### Dates and Times in Python
The Python world has a number of available representations of dates, times, deltas,
and timespans.
While the time series tools provided by Pandas tend to be the most useful for data science applications, it is helpful to see their relationship to other
packages used in Python.

In [2]:
import numpy as np
import pandas as pd

### Native Python dates and times: datetime and dateutil
Python’s basic objects for working with dates and times reside in the built-in date
time module. Along with the third-party dateutil module, you can use it to quickly
perform a host of useful functionalities on dates and times. For example, you can
manually build a date using the datetime type: